In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
# Given a m x n matrix  of m closing prices for n equities, returns an 1 x n 
# array of average closing prices
def compute_average(closing_prices_for_multiple_equities):
  sums = [0] * len(closing_prices_for_multiple_equities[0])
  for row in closing_prices_for_multiple_equities:
    for i in range(len(row)):
      sums[i] += row[i]

  avgs = [x/len(closing_prices_for_multiple_equities) for x in sums]
  return avgs

In [3]:
# Given an m x n matrix of m closing prices for n equities and n averages for n 
# equities, returns all of the difference of the closing prices and the averages
def compute_stocks_demeaned(closing_prices_matrix, average_closing_prices):
  m = len(closing_prices_matrix)
  n = len(closing_prices_matrix[0])
  demeaned = [[0 for i in range(n)] for j in range(m)]
  for i in range(len(closing_prices_matrix)):
    for j in range(len(closing_prices_matrix[i])):
      demeaned[i][j] = closing_prices_matrix[i][j] - average_closing_prices[j]

  return demeaned

In [5]:
# Given an m x n matrix of the demeaned closing prices for n equities, returns
# the associated covariance matrix calculated by demeaned transpose x demeaned.
def compute_covariance_matrix(demeaned):
    np_demeaned = np.array(demeaned)
    np_demeaned_transpose = np_demeaned.transpose()
    return np_demeaned_transpose.dot(np_demeaned)

In [4]:
def estimated_portfolio_risk_based_on_stddev (asset_weights, covariance_matrix):
  Wt = np.asmatrix(asset_weights)
  W = Wt.transpose()
  return np.sqrt(Wt.dot(covariance_matrix).dot(W))

In [6]:
def expected_portfolio_return(average_closing_prices, asset_weights):
  M = np.array(average_closing_prices)
  W = np.array(asset_weights)
  return M.dot(W)

In [7]:
def sharpe_ratio(expected_return, portfolio_std_dev, risk_free_rate):
  return (expected_return - risk_free_rate)/portfolio_std_dev

In [8]:
# Given array of equities, returning dataframe with historical data dating back 
def get_closing_price_historical_data(equities, period, interval):
  data = {}
  for x in equities:
    ticker = yf.Ticker(x)
    history = ticker.history(period= period, interval= interval)['Close']
    data[x] = history
  return data

In [10]:
senbet = pd.read_excel('Senbet Portfolio.xlsx')
senbet

,$,Allocation,Shares
0,MCD,0.0211,142
1,AMZN.O,0.0246,12
2,FIVE.O,0.0213,169
3,TJX,0.0197,451
4,DAN,0.0266,1650
5,PG,0.0212,236
6,STZ,0.0146,97
7,EL,0.0204,106
8,COST.O,0.0201,86
9,ZTS,0.0270,259


In [11]:
trimmed_senbet = senbet
trimmed_senbet.columns = ['Ticker', 'Allocation', 'Shares']
trimmed_senbet['Allocation'] *= 100
print(trimmed_senbet)

                Ticker  Allocation  Shares
0                  MCD        2.11     142
1               AMZN.O        2.46      12
2               FIVE.O        2.13     169
3                  TJX        1.97     451
4   DAN                       2.66    1650
5                   PG        2.12     236
6                  STZ        1.46      97
7                   EL        2.04     106
8               COST.O        2.01      86
9                  ZTS        2.70     259
10                 SYK        2.42     150
11              ICLR.O        2.25     173
12                 BMY        2.48     593
13                 DGX        2.38     280
14              ABBV.K        2.82     394
15                 HON        2.96     206
16                 RTX        2.37     464
17                  WM        2.64     309
18              AAPL.O        2.28     282
19              MSFT.O        2.68     172
20                   V        3.31     236
21              CSCO.O        2.46     718
22         

In [12]:
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.O', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.K', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace(' ', ''), axis = 1)
trimmed_senbet

,Ticker,Allocation,Shares
0,MCD,2.11,142
1,AMZN,2.46,12
2,FIVE,2.13,169
3,TJX,1.97,451
4,DAN,2.66,1650
5,PG,2.12,236
6,STZ,1.46,97
7,EL,2.04,106
8,COST,2.01,86
9,ZTS,2.70,259


In [14]:
asset_weights = list(trimmed_senbet['Allocation'])
asset_weights

[2.11,
 2.46,
 2.13,
 1.97,
 2.6599999999999997,
 2.12,
 1.46,
 2.04,
 2.01,
 2.7,
 2.42,
 2.25,
 2.48,
 2.3800000000000003,
 2.82,
 2.96,
 2.37,
 2.64,
 2.2800000000000002,
 2.68,
 3.3099999999999996,
 2.46,
 2.32,
 2.8400000000000003,
 2.59,
 2.5,
 2.02,
 2.08,
 1.09,
 1.38,
 3.0,
 3.08,
 2.39,
 3.0,
 1.38,
 1.3599999999999999,
 1.6199999999999999,
 0.98,
 1.5599999999999998,
 1.22,
 2.1999999999999997,
 2.16,
 2.21,
 2.2399999999999998,
 2.22]

In [16]:
historical_data = get_closing_price_historical_data(trimmed_senbet['Ticker'], '179d', '1d')

{'MCD': Date
 2020-07-22    195.097626
 2020-07-23    194.046616
 2020-07-24    195.195847
 2020-07-27    197.680984
 2020-07-28    192.759842
                  ...    
 2021-03-31    224.139999
 2021-04-01    225.210007
 2021-04-05    228.850006
 2021-04-06    231.910004
 2021-04-07    232.610001
 Name: Close, Length: 179, dtype: float64,
 'AMZN': Date
 2020-07-22    3099.909912
 2020-07-23    2986.550049
 2020-07-24    3008.909912
 2020-07-27    3055.209961
 2020-07-28    3000.330078
                  ...     
 2021-03-31    3094.080078
 2021-04-01    3161.000000
 2021-04-05    3226.729980
 2021-04-06    3223.820068
 2021-04-07    3279.389893
 Name: Close, Length: 179, dtype: float64,
 'FIVE': Date
 2020-07-22    109.089996
 2020-07-23    109.139999
 2020-07-24    110.180000
 2020-07-27    111.599998
 2020-07-28    110.480003
                  ...    
 2021-03-31    190.789993
 2021-04-01    195.619995
 2021-04-05    199.149994
 2021-04-06    197.429993
 2021-04-07    194.660004
 Nam

In [82]:
portfolio = pd.DataFrame(historical_data)
# historical_df =historical_df.dropna(axis = 0)
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-22,195.097626,3099.909912,109.089996,53.238632,12.717729,123.909447,177.381836,194.366638,317.993225,144.704330,...,69.065224,727.262573,165.661942,101.015358,15.965771,95.233887,202.853455,95.515427,37.716969,34.220284
2020-07-23,194.046616,2986.550049,109.139999,53.188820,12.628097,124.711136,176.897614,192.905685,315.872009,143.667252,...,69.200691,728.419434,165.299011,99.192329,16.188866,95.025688,199.918991,95.644043,38.295872,33.586208
2020-07-24,195.195847,3008.909912,110.180000,52.690704,12.478710,124.513428,177.737610,190.570129,315.552399,143.816833,...,69.023354,729.111633,165.328445,98.226616,15.878473,95.491646,198.392654,94.318268,38.129070,33.189907
2020-07-27,197.680984,3055.209961,111.599998,51.883759,12.667933,124.869293,176.788879,191.653427,317.315216,146.509277,...,68.378075,745.535156,167.103836,97.083527,15.839675,93.726959,199.918991,94.773384,37.559978,33.397961
2020-07-28,192.759842,3000.330078,110.480003,51.584888,12.598219,126.411377,177.411484,191.414917,317.286163,145.571899,...,68.971642,744.081604,170.468277,95.142258,15.393486,92.120888,198.540359,93.417923,37.432430,33.744724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-31,224.139999,3094.080078,190.789993,66.150002,24.330000,135.429993,228.000000,290.850006,352.480011,157.479996,...,75.610001,679.590027,164.300003,158.820007,21.610001,111.680000,327.000000,141.440002,61.889999,44.500000
2021-04-01,225.210007,3161.000000,195.619995,65.980003,24.570000,134.279999,229.619995,292.329987,354.940002,157.889999,...,75.769997,678.000000,169.720001,164.600006,21.730000,114.199997,327.640015,144.570007,62.430000,44.750000
2021-04-05,228.850006,3226.729980,199.149994,67.760002,24.610001,136.449997,230.279999,295.269989,360.820007,158.410004,...,76.779999,690.239990,170.690002,152.179993,21.410000,115.110001,323.540009,146.059998,62.750000,45.400002


In [55]:
portfolio.shape

(179, 45)

In [83]:
for index, x in trimmed_senbet.iterrows():
    portfolio[x.Ticker]*=x.Shares

In [84]:
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-22,27703.862854,37198.918945,18436.209381,24010.623123,20984.252214,29242.629425,17206.038086,20602.863647,27347.417358,37478.421585,...,18785.740845,26181.452637,14909.574738,14950.272980,13618.802425,28284.464355,20285.345459,22541.640686,20631.181789,25767.873482
2020-07-23,27554.619415,35838.600586,18444.659897,23988.157768,20836.359358,29431.828064,17159.068512,20448.002655,27164.992798,37209.818161,...,18822.588013,26223.099609,14876.911011,14680.464752,13809.102409,28222.629387,19991.899109,22571.994141,20947.841839,25290.414883
2020-07-24,27717.810211,36106.918945,18620.420052,23763.507660,20589.871788,29385.168945,17240.548157,20200.433716,27137.506287,37248.559875,...,18774.352173,26248.018799,14879.560089,14537.539154,13544.337709,28361.018806,19839.265442,22259.111206,20856.601444,24992.000027
2020-07-27,28070.699799,36662.519531,18860.399742,23399.575104,20902.088642,29469.153198,17148.521301,20315.263275,27289.108582,37945.902832,...,18598.836304,26839.265625,15039.345245,14368.361938,13511.242732,27836.906891,19991.899109,22366.518646,20545.308231,25148.664379
2020-07-28,27371.897552,36003.960938,18671.120567,23264.784695,20787.061214,29833.084961,17208.913925,20289.981201,27286.610046,37703.121948,...,18760.286499,26786.937744,15342.144928,14081.054138,13130.643578,27359.903664,19854.035950,22046.629822,20475.539356,25409.777378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-31,31827.879913,37128.960938,32243.508865,29833.650688,40144.499874,31961.478271,22116.000000,30830.100647,30313.280945,40787.318893,...,20565.920166,24465.240967,14787.000275,23505.361084,18433.330521,33168.960091,32700.000000,33379.840576,33853.829666,33508.500000
2021-04-01,31979.820953,37932.000000,33059.779175,29756.981514,40540.499496,31690.079712,22273.139526,30986.978577,30524.840210,40893.509842,...,20609.439087,24408.000000,15274.800110,24360.800903,18535.689610,33917.399094,32764.001465,34118.521729,34149.210167,33696.750000
2021-04-05,32496.700867,38720.759766,33656.348969,30559.760963,40606.501007,32202.199280,22337.159882,31298.618835,31030.520630,41028.190948,...,20884.159668,24848.639648,15362.100220,22522.638916,18262.729870,34187.670181,32354.000854,34470.159424,34324.250000,34186.201149


In [58]:
portfolio["PortValue"] = portfolio.sum(axis=1)
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM,PortValue
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-22,27703.862854,37198.918945,18436.209381,24010.623123,20984.252214,29242.629425,17206.038086,20602.863647,27347.417358,37478.421585,...,26181.452637,14909.574738,14950.272980,13618.802425,28284.464355,20285.345459,22541.640686,20631.181789,25767.873482,1.221269e+06
2020-07-23,27554.619415,35838.600586,18444.659897,23988.157768,20836.359358,29431.828064,17159.068512,20448.002655,27164.992798,37209.818161,...,26223.099609,14876.911011,14680.464752,13809.102409,28222.629387,19991.899109,22571.994141,20947.841839,25290.414883,1.209864e+06
2020-07-24,27717.810211,36106.918945,18620.420052,23763.507660,20589.871788,29385.168945,17240.548157,20200.433716,27137.506287,37248.559875,...,26248.018799,14879.560089,14537.539154,13544.337709,28361.018806,19839.265442,22259.111206,20856.601444,24992.000027,1.198857e+06
2020-07-27,28070.699799,36662.519531,18860.399742,23399.575104,20902.088642,29469.153198,17148.521301,20315.263275,27289.108582,37945.902832,...,26839.265625,15039.345245,14368.361938,13511.242732,27836.906891,19991.899109,22366.518646,20545.308231,25148.664379,1.207626e+06
2020-07-28,27371.897552,36003.960938,18671.120567,23264.784695,20787.061214,29833.084961,17208.913925,20289.981201,27286.610046,37703.121948,...,26786.937744,15342.144928,14081.054138,13130.643578,27359.903664,19854.035950,22046.629822,20475.539356,25409.777378,1.202633e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-31,31827.879913,37128.960938,32243.508865,29833.650688,40144.499874,31961.478271,22116.000000,30830.100647,30313.280945,40787.318893,...,24465.240967,14787.000275,23505.361084,18433.330521,33168.960091,32700.000000,33379.840576,33853.829666,33508.500000,1.510228e+06
2021-04-01,31979.820953,37932.000000,33059.779175,29756.981514,40540.499496,31690.079712,22273.139526,30986.978577,30524.840210,40893.509842,...,24408.000000,15274.800110,24360.800903,18535.689610,33917.399094,32764.001465,34118.521729,34149.210167,33696.750000,1.534287e+06
2021-04-05,32496.700867,38720.759766,33656.348969,30559.760963,40606.501007,32202.199280,22337.159882,31298.618835,31030.520630,41028.190948,...,24848.639648,15362.100220,22522.638916,18262.729870,34187.670181,32354.000854,34470.159424,34324.250000,34186.201149,1.547504e+06


In [78]:
historical_nparray = historical_df.to_numpy()
senbet_avg_closing_prices = compute_average(historical_nparray)
print(senbet_avg_closing_prices)

[210.3414534311446, 3173.13062556837, 149.98423274610408, 60.34365055548451, 16.84000000000003, 133.50584669970962, 200.33053895657656, 239.3456143979673, 347.05422860604745, 158.08298674083892, 219.1330674590257, 192.3600537335431, 24.318888901402712, 120.54372684539311, 97.6873151183759, 184.71126972178303, 65.88861023433624, 113.85109347388858, 119.02332293798052, 218.23738235392898, 204.54609373385313, 43.4979782306328, 64.41603185260107, 349.053542041274, 124.91568743993365, 147.6291533374282, 498.1202105163897, 24.226385671625692, 13.672751353531288, 1726.4304205677497, 267.38365084532074, 129.35814856473732, 64.46299428788443, 148.4242099176639, 73.38982788974015, 725.0172729492188, 164.2409406389509, 111.56228048334677, 18.29789378151061, 104.72390452389995, 243.765465307488, 112.9928893639297, 46.47722827568256]


In [17]:
senbet_demeaned = compute_stocks_demeaned(historical_nparray, senbet_avg_closing_prices)

In [18]:
senbet_covariance_matrix = compute_covariance_matrix(senbet_demeaned)

In [19]:
senbet_exp_risk= estimated_portfolio_risk_based_on_stddev(asset_weights, senbet_covariance_matrix)
print(senbet_exp_risk)

[[14720.3919503]]


In [20]:
senbet_exp_return = expected_portfolio_return(senbet_avg_closing_prices, asset_weights)
print(senbet_exp_return)

29504.396676434328


In [21]:
sharpe_ratio(senbet_exp_return, senbet_exp_risk, .01485*1535893.08)

matrix([[0.45490531]])

In [22]:
apex = pd.DataFrame(index=['BATRA','PD','ACI'], columns=['Allocation'], data = [.0705, .1014, .0798])

In [23]:
apex

,Allocation
BATRA,0.0705
PD,0.1014
ACI,0.0798


In [24]:
apex_closing_prices = get_closing_price_historical_data(list(apex.index.values), '6mo', '1d')

In [25]:
apex_historical_df = pd.DataFrame(apex_closing_prices)

In [26]:
apex_historical_nparray = apex_historical_df.to_numpy()
apex_avg_closing_prices = compute_average(apex_historical_nparray)
print(apex_avg_closing_prices)

[26.315600036621095, 39.85192008972168, 16.30978335571289]


In [27]:
apex_demeaned = compute_stocks_demeaned(apex_historical_nparray, apex_avg_closing_prices)

In [28]:
apex_covariance_matrix = compute_covariance_matrix(apex_demeaned)

In [29]:
apex_asset_weights = list(apex['Allocation'])
print(apex_asset_weights)

[0.0705, 0.1014, 0.0798]


In [30]:
apex_exp_risk= estimated_portfolio_risk_based_on_stddev(apex_asset_weights, apex_covariance_matrix)
print(apex_exp_risk)

[[12.27824733]]


In [31]:
apex_exp_return = expected_portfolio_return(apex_avg_closing_prices, apex_asset_weights)
print(apex_exp_return)

7.197755211465454


In [32]:
sharpe_ratio(apex_exp_return, apex_exp_risk, 1.485)

matrix([[0.46527449]])

In [59]:
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM,PortValue
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-22,27703.862854,37198.918945,18436.209381,24010.623123,20984.252214,29242.629425,17206.038086,20602.863647,27347.417358,37478.421585,...,26181.452637,14909.574738,14950.272980,13618.802425,28284.464355,20285.345459,22541.640686,20631.181789,25767.873482,1.221269e+06
2020-07-23,27554.619415,35838.600586,18444.659897,23988.157768,20836.359358,29431.828064,17159.068512,20448.002655,27164.992798,37209.818161,...,26223.099609,14876.911011,14680.464752,13809.102409,28222.629387,19991.899109,22571.994141,20947.841839,25290.414883,1.209864e+06
2020-07-24,27717.810211,36106.918945,18620.420052,23763.507660,20589.871788,29385.168945,17240.548157,20200.433716,27137.506287,37248.559875,...,26248.018799,14879.560089,14537.539154,13544.337709,28361.018806,19839.265442,22259.111206,20856.601444,24992.000027,1.198857e+06
2020-07-27,28070.699799,36662.519531,18860.399742,23399.575104,20902.088642,29469.153198,17148.521301,20315.263275,27289.108582,37945.902832,...,26839.265625,15039.345245,14368.361938,13511.242732,27836.906891,19991.899109,22366.518646,20545.308231,25148.664379,1.207626e+06
2020-07-28,27371.897552,36003.960938,18671.120567,23264.784695,20787.061214,29833.084961,17208.913925,20289.981201,27286.610046,37703.121948,...,26786.937744,15342.144928,14081.054138,13130.643578,27359.903664,19854.035950,22046.629822,20475.539356,25409.777378,1.202633e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-31,31827.879913,37128.960938,32243.508865,29833.650688,40144.499874,31961.478271,22116.000000,30830.100647,30313.280945,40787.318893,...,24465.240967,14787.000275,23505.361084,18433.330521,33168.960091,32700.000000,33379.840576,33853.829666,33508.500000,1.510228e+06
2021-04-01,31979.820953,37932.000000,33059.779175,29756.981514,40540.499496,31690.079712,22273.139526,30986.978577,30524.840210,40893.509842,...,24408.000000,15274.800110,24360.800903,18535.689610,33917.399094,32764.001465,34118.521729,34149.210167,33696.750000,1.534287e+06
2021-04-05,32496.700867,38720.759766,33656.348969,30559.760963,40606.501007,32202.199280,22337.159882,31298.618835,31030.520630,41028.190948,...,24848.639648,15362.100220,22522.638916,18262.729870,34187.670181,32354.000854,34470.159424,34324.250000,34186.201149,1.547504e+06


In [60]:
portvalue = list(portfolio['PortValue'])

In [85]:
import statistics

In [86]:
pct_change = []
for i in range(1, len(portvalue)):
    curr = portvalue[i]
    yester = portvalue[i-1]
    delta = curr-yester
    pct_change.append(delta/curr * 100)

In [87]:
avg_pct_change = statistics.mean(pct_change)
stdev_pct_change = statistics.stdev(pct_change)
print(avg_pct_change, stdev_pct_change, avg_pct_change/stdev_pct_change)

0.12713620064970435 1.0585996537432032 0.12009847178784856


In [88]:
for sym in portfolio.columns:
    data = list(portfolio[sym])
    pct_change = []
    for i in range(1, len(data)):
        curr = data[i]
        yester = data[i-1]
        delta = curr-yester
        pct_change.append(delta/curr * 100)
    pct_change.insert(0, 0)
    portfolio[sym] = pct_change

portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-22,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-07-23,-0.541628,-3.795679,0.045816,-0.093652,-0.709783,0.642837,-0.273730,-0.757340,-0.671543,-0.721862,...,0.195760,0.158818,-0.219560,-1.837873,1.378076,-0.219097,-1.467826,0.134474,1.511660,-1.887903
2020-07-24,0.588758,0.743122,0.943911,-0.945358,-1.197130,-0.158785,0.472605,-1.225562,-0.101286,0.104009,...,-0.256924,0.094937,0.017803,-0.983149,-1.954800,0.487956,-0.769351,-1.405640,-0.437465,-1.194042
2020-07-27,1.257146,1.515446,1.272400,-1.555296,1.493711,0.284990,-0.536646,0.565238,0.555541,1.837729,...,-0.943693,2.202917,1.062448,-1.177429,-0.244944,-1.882795,0.763478,0.480215,-1.515155,0.622953
2020-07-28,-2.552992,-1.829128,-1.013754,-0.579375,-0.553361,1.219893,0.350938,-0.124604,-0.009157,-0.643928,...,0.860596,-0.195348,1.973646,-2.040386,-2.898557,-1.743439,-0.694384,-1.450965,-0.340743,1.027608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-31,-0.374764,1.253686,0.151996,-0.559327,-1.356350,-0.014771,-2.609648,-1.471548,0.774515,0.279402,...,1.838380,0.698951,-1.199027,-1.133351,-0.555294,0.402934,-1.532113,-1.145359,-0.791730,-1.910109
2021-04-01,0.475115,2.117049,2.469074,-0.257651,0.976800,-0.856415,0.705511,0.506270,0.693072,0.259677,...,0.211160,-0.234517,3.193494,3.511542,0.552227,2.206652,0.195341,2.165044,0.864970,0.558659
2021-04-05,1.590561,2.037046,1.772533,2.626917,0.162539,1.590325,0.286609,0.995700,1.629623,0.328265,...,1.315450,1.773295,0.568282,-8.161397,-1.494627,0.790551,-1.267233,1.020122,0.509960,1.431721


In [89]:
senbet_port = portfolio.to_numpy()
senbet_avg_returns = compute_average(senbet_port)
print(senbet_avg_returns)

[0.09239359861747862, 0.011687206484009173, 0.2976827231015651, 0.11384797864388435, 0.3063949789927734, 0.05011737027493199, 0.14230228268159265, 0.217858301949982, 0.06007457018184038, 0.03763042893965515, 0.12446672727306908, -0.004680875781625016, 0.02325920003316419, -0.013828219382013271, 0.04352071175081204, 0.18903405530342068, 0.09949756278808686, 0.11365006752312624, 0.1265187335577643, 0.08027292192583253, 0.04434044899664236, 0.05562775181297461, 0.29838340808641045, 0.15053391966328328, 0.03175540581053435, 0.14599722171659538, -0.07606839738235602, 0.05122720453110951, 0.20194727942001067, 0.41053965854526064, 0.18221429775995174, 0.12146953868414499, 0.1884300918087621, 0.15349197679526466, 0.030348593478685956, 0.050107365367051654, -0.04842914931960268, 0.005835203713245938, 0.1867509088090611, 0.12890037388677436, 0.09521325272284864, 0.2505484717178994, 0.21406438745705006, 0.26306607746844946, 0.13583811981787666]


In [90]:
senbet_demeaned = compute_stocks_demeaned(senbet_port, senbet_avg_returns)

In [91]:
senbet_covmat = compute_covariance_matrix(senbet_demeaned)

In [92]:
senbet_asset_weights = list(trimmed_senbet['Allocation'])

In [97]:
estimated_portfolio_risk_based_on_stddev(senbet_asset_weights, senbet_covmat)

matrix([[1421.36038569]])

In [99]:
expected_portfolio_return(asset_weights=senbet_asset_weights, average_closing_prices = senbet_avg_returns)

11.937223616913187

In [101]:
(11.937223616913187-1.4) / 1421.36038569

0.0074134777660894826